In [12]:
import datasets
from transformers import DistilBertModel, DistilBertTokenizer, Trainer
import torch
import torch.nn as nn

import pandas as pd
import pytorch_lightning as pl

from datasets import Dataset
from torchmetrics import functional as F

# Build Model

In [57]:
class DistilFakeBert(nn.Module):
    def __init__(self):
        super(DistilFakeBert, self).__init__()
        
        self.distil_bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(768, 256)
        self.relu = nn.ReLU()
        self.output = nn.Linear(256, 2)
        
    def forward(self, input_ids, attention_mask):
        embeddings = self.distil_bert(
            input_ids, attention_mask=attention_mask
        )[0][:, 0]
        embeddings = self.fc1(embeddings)
        embeddings = self.relu(embeddings)
        embeddings = self.dropout(embeddings)
        output = self.output(embeddings)
        
        return output

In [58]:
model = DistilFakeBert()
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [59]:
model(**inputs)

tensor([[-0.1486, -0.0224]], grad_fn=<AddmmBackward>)

In [40]:
inputs = tokenizer.encode_plus(
    dataset[0]['text'], truncation=True, max_length=300,
    padding='max_length', return_tensors='pt', add_special_tokens=True
)

In [3]:
df = pd.read_csv('data_text/news.csv')

In [10]:
df = df.drop(df.columns[0], axis=1)

In [16]:
dataset = Dataset.from_pandas(df)